In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
def OneHot(input_dim=None, input_length=None):
    # Check if inputs were supplied correctly
    if input_dim is None or input_length is None:
        raise TypeError("input_dim or input_length is not set")

    # Helper method (not inlined for clarity)
    def _one_hot(x, num_classes):
        return keras.backend.one_hot(keras.backend.cast(x, 'uint8'),
                          num_classes=num_classes)

    # Final layer representation as a Lambda layer
    return layers.Lambda(_one_hot,
                  arguments={'num_classes': input_dim},
                  input_shape=(input_length,))

In [4]:
# Rounds:
# - Blind nil (no hand) -> outcomes[normal, blind]
# - Bid (hand) -> outcomes[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
# - Game x[1..13] -> outcomes[cards*]


training_model.summary()

training_model.compile(
    loss=tf.keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error"),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
)

Model: "spades1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bid_state_bids (InputLayer)     [(None, 4, 15)]      0                                            
__________________________________________________________________________________________________
bid_state_bags (InputLayer)     [(None, 10, 2)]      0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 60)           0           bid_state_bids[0][0]             
__________________________________________________________________________________________________
bid_state_hand (InputLayer)     [(None, 52)]         0                                            
____________________________________________________________________________________________

In [ ]:
qval = 1
generation = 1

def gen_samples(qval, generation):
    with concurrent.futures.ProcessPoolExecutor(max_workers=16) as executor:
		for d in executor.map(train, todos):
            yield d


In [5]:
qval = 1
generation = 1

inference_types = [tf.int32, tf.int32, tf.int32]
training_types = [tf.int32, tf.int32]
for i in range(13):
    inference_types += [tf.int32, tf.int32, tf.int32, tf.int32]
    training_types.append(tf.int32)
    
def to_keras():
    for i in range(100):
        data = np.load(f'data/q{qval}/gen{generation:03}/{i:02}.npy', allow_pickle=True)
        for d in data:
            inputs = d['input']
            score = d['training']['score']
            bid_state_bids = inputs['bids']['bids']
            bid_state_hand = inputs['bids']['hand']
            bid_state_bags = inputs['bids']['bags']

            inference_inputs = [bid_state_bids, bid_state_hand, bid_state_bags]
            for i in range(13):
                roundname='round' + str(i)
                seen = inputs[roundname]['seen']
                hand = inputs[roundname]['hand']
                played = inputs[roundname]['played']
                todo = inputs[roundname]['todo']
                inference_inputs = inference_inputs + [seen, hand, played, todo]

            training_inputs = []
            training_inputs.append(d['training']['bids'])
            training_inputs.append(score)
            for i in range(13):
                training_inputs.append(d['training']['round' + str(i)])
            outputs = [score, np.stack([score] * 13)]

            yield tuple([np.expand_dims(np.array(i), axis=0) for i in inference_inputs + training_inputs]), tuple([np.expand_dims(np.array(i), axis=0) for i in outputs])

dataset = tf.data.Dataset.from_generator(lambda: to_keras(), output_types=(
    tuple(inference_types + training_types),
    (tf.int32, tf.int32)
))
dataset = dataset.unbatch()
dataset = dataset.cache(f'data/q{qval}/data.cache')

In [6]:
trainingset = dataset.shuffle(10000)
trainingset = trainingset.batch(1024)
print(trainingset)
history = training_model.fit(trainingset, batch_size=1024, epochs=5)

<BatchDataset shapes: ((<unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>, <unknown>), (<unknown>, <unknown>)), types: ((tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, tf.int32, t

In [15]:
inference_model.save(f'models/q{qval}/gen{generation:03}.model')

INFO:tensorflow:Assets written to: models/q2/gen001.model\assets


INFO:tensorflow:Assets written to: models/q2/gen001.model\assets
